___

# M915 - Συστήματα Κατανόησης και Παραγωγής Κειμένου <br> <span style="font-size:6mm;"> Assignment 1 </span> <br><br> <span style="font-size:5mm;"> Kylafi Christina-Theano </span> <br> <span style="font-size:4mm;"> LT1200012 </span>
---
---

## Imports
---

In [4]:
from heapq import heapify
import os
import json
from typing import Counter
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

import math

import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer, LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag

from sklearn.feature_extraction.text import CountVectorizer
import sklearn.model_selection
from sklearn.model_selection import RepeatedKFold, cross_val_score, GridSearchCV
import sklearn.preprocessing as preproc
from sklearn.feature_extraction import text
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, ElasticNet, ElasticNetCV, LassoLars, SGDRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

from IPython.display import FileLink, FileLinks

from tqdm import tqdm

!pip install rouge-score
from rouge_score import rouge_scorer

import psutil
import gc
from collections import Counter

# Add Data -> "Summarizer Data" / "summarizer-data" -> "Add"
df_dir="/kaggle/input/summarizer-data"
data_dir= "/kaggle/working/Data/DataFrames"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=c0374d71be1efd5aa25fcc07f77fef3808b1a2c24bac26242a90ab9975b72397
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


## Functions
---

In [5]:
# create dataframes of the json datasets (train, dev & test)
def json_to_df(json_path,data_type):
    data=[]
    for ln in open(json_path,"r"):
        obj = json.loads(ln)
        data.append(obj)
    df=pd.DataFrame(data)
    
    cols=[ "title",	"date",	"text",	"summary", "compression", "coverage", "density", "compression_bin", "coverage_bin"]
    df=df.loc[df.density_bin=="extractive"].reset_index()
    
    # save the dataframe and the summaries as they are for future use
    df.to_csv(f"/kaggle/working/Data{os.sep}DataFrames{os.sep}{data_type}_set.csv", header=True, index=False )
    df["summary"].to_csv(f"/kaggle/working/Data{os.sep}DataFrames{os.sep}{data_type}_summaries.csv", header=True, index=False )
    
    return df


# text processing functions
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'd've": "he would have", "he'll": "he will", "he's": "he is", "how'd": "how did", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'll": "i will", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'll": "it will", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "must've": "must have", "mustn't": "must not", "needn't": "need not", "oughtn't": "ought not", "shan't": "shall not", "sha'n't": "shall not", "she'd": "she would", "she'll": "she will", "she's": "she is", "should've": "should have", "shouldn't": "should not", "that'd": "that would", "that's": "that is", "there'd": "there had", "there's": "there is", "they'd": "they would", "they'll": "they will", "they're": "they are", "they've": "they have", "wasn't": "was not", "we'd": "we would", "we'll": "we will", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what're": "what are", "what's": "what is", "what've": "what have", "where'd": "where did", "where's": "where is", "who'll": "who will", "who's": "who is", "won't": "will not", "wouldn't": "would not", "you'd": "you would", "you'll": "you will", "you're": "you are" }


# text cleaning process
def sentence_cleaning(text, remove_stopwords = True, sub_contractions=True, stemming=True):
    global pbar_cleaning
    pbar_cleaning.update(1)
    
    # Convert words to lower case
    text = text.lower()
    toks = word_tokenize(text)
      
    # Replace contractions with their longer forms 
    if sub_contractions:
        new_text = []
        for word in toks:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
    
    text = " ".join(new_text)

    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)

    toks_clean = word_tokenize(text)
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        toks_clean = [w for w in toks_clean if not w in stops]

    
    if stemming: 
        stemmer=SnowballStemmer(language="english")
        toks_clean=[ stemmer.stem(w) for w in toks_clean ]

    text = " ".join(toks_clean)
    
    return text, toks_clean


# score sentences (RougeL fmeasure) with respect to gold summaries 
# Source: [SummaRuNNer: a recurrent neural network based sequence model for extractive summarization of documents](https://arxiv.org/abs/1611.04230)
def rouge_scoring(sentence,summary,sc_type="rougeL",score="fmeasure"):
    global pbar
    pbar.update(1)
    r_scorer=rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"])
    r_L=r_scorer.score(summary,sentence)
    score_ind={"precision":0, "recall":1, "fmeasure":2}
    
    return r_L[sc_type][score_ind[score]]


def text_processing(df,data_type,df_dir,sc_type="rougeL"):
    global pbar
    cols=["sentence", "summary", "text", "text_id"] 
    
    df["summary"].to_csv(os.path.join(df_dir,f"{data_type}_data_{sc_type}_summaries_grouped.csv"), header=True, index=False)

    # sentence split 
    sentences=[ sent_tokenize(t) for t in df["text"].values ]

    summaries=df["summary"].values
    sent_sum_text=[ [ s,summary,t,i  ] for i, (s_list,summary,t) in enumerate(zip( sentences, summaries, df["text"] )) for s in s_list ]
    new_df=pd.DataFrame(sent_sum_text, columns=cols)
    
    # add a column indicating if sentence was chosen to be in the summary
    new_df["chosen"]= 0
    ind = new_df[[ s in t for s,t in zip( new_df["sentence"], new_df["summary"] ) ]].index
    new_df.loc[ind,"chosen"]=1
    del new_df["text"]

    # labels - RougeL scores
    pbar = tqdm(total=new_df.shape[0] )
    new_df["rougeL"]= new_df.apply(lambda row: rouge_scoring(row["sentence"],row["summary"], sc_type=sc_type, score="fmeasure" ), axis=1)

    new_df["summary"].to_csv(os.path.join(df_dir,f"{data_type}_data_{sc_type}_summaries.csv"), header=True, index=False)
    del new_df["summary"]
    
    # save dataframe with scoring/labels
    new_df.to_csv(os.path.join(df_dir,f"{data_type}_data_{sc_type}.csv"), header=True, index=False)

    return new_df


def add_chosen_text_id(df):
    cols=["sentence", "summary", "text"] 

    # sentence split 
    sentences=[ sent_tokenize(t) for t in df["text"].values ]
    summaries=df["summary"].values
    
    sent_sum_text=[ [ s,summary,t  ] for s_list,summary,t in zip( sentences, summaries, df["text"] ) for s in s_list  ]
    new_df=pd.DataFrame(sent_sum_text, columns=cols)
    
    new_df["text_id"]=new_df["text"].factorize()[0]
    new_df["chosen"]= 0
    ind = new_df[[ s in t for s,t in zip( new_df["sentence"], new_df["summary"] ) ]].index
    new_df.loc[ind,"chosen"]=1
    
    del new_df["text"]
    del new_df["summary"]
    del new_df["sentence"]
    
    return new_df


def text_proc_labels(data_dir,df_dir="/kaggle/input/summarizer-data/"):
#     train_df = pd.read_csv(os.path.join(df_dir,"train_set.csv"))
    dev_df = pd.read_csv(os.path.join(df_dir,"dev_set.csv"))
    test_df = pd.read_csv(os.path.join(df_dir,"test_set.csv"))

    dev_data=None
    test_data=None
    
#     train_df1, train_df2, train_df3, train_df4 = np.array_split(train_df, 4)
    splits=4
    train_data_list=[]
    for i in range(splits):
        train_data_type=f"train{i+1}"
        train_df = pd.read_csv(os.path.join(df_dir,f"{train_data_type}_set.csv"))
        train_data = text_processing(train_df,f"train{i+1}",data_dir,"rougeL")
        train_data_list.append(train_data)
    
    test_data = text_processing(test_df,"test",data_dir,"rougeL")
    dev_data = text_processing(dev_df,"dev",data_dir,"rougeL")
    
    
    return train_data_list, dev_data, test_data


def df_add_tid(data_dir, df_dir):
    sc_type="rougeL"
    data_labels_dir="/kaggle/input/summarizer-data"

    data_type="train"
    train_df = pd.read_csv(os.path.join(df_dir,f"{data_type}_set.csv"))
    splits=4
    for i,df in enumerate(np.array_split(train_df, 4)):
        if i==2:
            data_type=f"train{i+1}"
            print(data_type+"\n")
            train_ch_tid_df=add_chosen_text_id(df)
            train_rougeL=pd.read_csv(os.path.join(data_labels_dir,f"{data_type}_data_{sc_type}.csv"))
            print(train_rougeL.columns, train_rougeL.shape)
            train_rougeL["text_id"]=train_ch_tid_df["text_id"]
            train_rougeL["chosen"]=train_ch_tid_df["chosen"]
            print(train_rougeL.columns, train_rougeL.shape)
            train_rougeL.to_csv(os.path.join(data_dir,f"{data_type}_data_{sc_type}_tid.csv"), header=True, index=False)

    data_type="dev" 
    dev_df = pd.read_csv(os.path.join(df_dir,f"{data_type}_set.csv"))
    dev_ch_tid_df=add_chosen_text_id(dev_df)
    dev_rougeL=pd.read_csv(os.path.join(data_labels_dir,f"{data_type}_data_{sc_type}.csv"))
    print(dev_rougeL.columns, dev_rougeL.shape)
    dev_rougeL["text_id"]=dev_ch_tid_df["text_id"]
    dev_rougeL["chosen"]=dev_ch_tid_df["chosen"]
    print(dev_rougeL.columns, dev_rougeL.shape)
    dev_rougeL.to_csv(os.path.join(data_dir,f"{data_type}_data_{sc_type}_tid.csv"), header=True, index=False)

    data_type="test"
    test_df = pd.read_csv(os.path.join(df_dir,f"{data_type}_set.csv"))
    test_ch_tid_df=add_chosen_text_id(test_df)
    test_rougeL=pd.read_csv(os.path.join(data_labels_dir,f"{data_type}_data_{sc_type}.csv"))
    print(test_rougeL.columns, test_rougeL.shape)
    test_rougeL["text_id"]=test_ch_tid_df["text_id"]
    test_rougeL["chosen"]=test_ch_tid_df["chosen"]
    print(test_rougeL.columns, test_rougeL.shape)
    test_rougeL.to_csv(os.path.join(data_dir,f"{data_type}_data_{sc_type}_tid.csv"), header=True, index=False)


# sentence statistics 
def data_stats(df):
    groups = df.groupby("chosen")
    print(f"DataFrame shape: {df.shape}\n\n")
    print(groups.describe()["rougeL"])

    
# feature variable: ratio of thematic words in sentence
def thematic_ratio(them_words, word_list):
    them_occ= sum( [ word_list.count(w) for w in set(them_words)&set(word_list)])
    them_ratio=them_occ/len(word_list)
    return them_ratio

# feature variable: sentence position - "By this, we get a high feature value towards the beginning and ending of the document, and a progressively decremented value towards the middle" as mentioned in the paper
def s_position(t_position,tot_sent):
    N=tot_sent
    th=0.2*N
    min_p= th*N
    max_p= 2*th*N
    
    if t_position==tot_sent or t_position==1:
        pos=1.0
    else: 
        pos=math.cos((t_position - min_p)*((1/max_p) - min_p))
        
    return pos
    
# feature variable (failed experiment) - timeout, too much time needed for pos tagging - left for future work     
def prop_nouns(tokens):
    if type(tokens)!=list:
        tokens=eval(tokens)
    pos= nltk.pos_tag(tokens)
    tags_count=Counter(tag for _, tag in pos if tag=="NNP" or tag=="NNPS")
    return tags_count["NNP"]+tags_count["NNPS"] 


# create sentence features
def feature_df(df, data_dir, data_type):
    global pbar_cleaning
    feat_df=pd.DataFrame()
    
    # tokenize sentences
    pbar_cleaning=tqdm(total=df.shape[0], leave=True)
    df["tokens"] = df["sentence"].apply(lambda x: sentence_cleaning(x)[1]) 

    # create sentence features based on paper "Extractive Summarization using Deep Learning"
    # Source: [Extractive Summarization using Deep Learning](https://arxiv.org/pdf/1708.04439v1.pdf)
    # 1. thematic words ratio
    col = df.groupby("text_id")["tokens"].apply(sum)
    thematic_cols= pd.DataFrame({"text_id": col.index, "thematic": [ [ t[0] for t in Counter(x).most_common(10) ] for x in col ]})  
    df=df.join(thematic_cols["thematic"], on='text_id' )
    feat_df["thematic_ratio"] = df.apply(lambda row: thematic_ratio(row.thematic, row.tokens) if len(row.tokens)>0 else 0.0, axis=1)

    # 2. sentence position in the text
    feat_df["text_position"] = df.groupby("text_id").cumcount().add(1)
    df["text_position"]=feat_df["text_position"]
    df["tot_sent"] = df.groupby("text_id")["sentence"].transform(len)
    feat_df["s_position"] = df.apply(lambda row: s_position(row.text_position,row.tot_sent), axis=1)

    # 3. sentence length - threshold=3
    threshold=3
    feat_df["len"]= df["tokens"].apply(lambda x: 0 if len(x)<threshold else len(x))

    # 4. sentence position - paragraph relative
    feat_df['s_pos_par'] = feat_df["s_position"].values
    feat_df.loc[feat_df.s_pos_par!=1.0, 's_pos_par']=0.0

    # 5. numerals ratio
    feat_df["num_ratio"]=df["tokens"].apply(lambda x: sum( [ 1 for t in x if t.isnumeric() ] )/len(x) if len(x)>0 else 0 )

    # store sentence features
    feat_df.to_csv(os.path.join(data_dir,f"{data_type}_set_feats.csv"), header=True, index=False)

    return feat_df


def load_datasets(df_dir, sc_type="rougeL"):
    data_type="train1"
    print(f"\nLoading {data_type.capitalize()} Data . . ")
    # train_data=feature_df(train_data, data_dir, data_type)
    train_data=pd.read_csv(f"/kaggle/input/summarizer-data/{data_type}_set_feats.csv") 
    
    train_data_rougeLTid=pd.read_csv(os.path.join(df_dir,f"{data_type}_data_{sc_type}.csv"))
    train_data_rougeLTid["text_position"] = train_data_rougeLTid.groupby("text_id").cumcount().add(1)
    
    data_stats(train_data_rougeLTid)
    labels_train=train_data_rougeLTid["rougeL"].values
    
    train_data_feats=train_data
    train_data["sentence"]=train_data_rougeLTid["sentence"]

    train_data_feats=train_data_feats[["len", "s_position", "thematic_ratio", "s_pos_par", "num_ratio"]]


    data_type="dev"
    print(f"\n\nLoading {data_type.capitalize()} Data . . ")
    dev_data=pd.read_csv(f"/kaggle/input/summarizer-data/{data_type}_set_feats.csv") 
    
    dev_data_rougeLTid=pd.read_csv(os.path.join(df_dir,f"{data_type}_data_{sc_type}.csv"))
    dev_data_rougeLTid["text_position"] = dev_data_rougeLTid.groupby("text_id").cumcount().add(1)
    
    data_stats(dev_data_rougeLTid)
    labels_dev=dev_data_rougeLTid["rougeL"].values
    
    dev_data_feats=dev_data
    dev_data["sentence"]=dev_data_rougeLTid["sentence"]
    dev_data_feats=dev_data_feats[["len", "s_position", "thematic_ratio", "s_pos_par", "num_ratio"]]


    data_type="test"
    print(f"\n\nLoading {data_type.capitalize()} Data . . ")
    test_data=pd.read_csv(f"/kaggle/input/summarizer-data/{data_type}_set_feats.csv") 
    
    test_data_rougeLTid=pd.read_csv(os.path.join(df_dir,f"{data_type}_data_{sc_type}.csv"))
    test_data_rougeLTid["text_position"] = test_data_rougeLTid.groupby("text_id").cumcount().add(1)
    
    data_stats(test_data_rougeLTid)
    labels_test=test_data_rougeLTid["rougeL"].values
    
    test_data_feats=test_data
    test_data["sentence"]=test_data_rougeLTid["sentence"]
    test_data_feats=test_data_feats[["len", "s_position", "thematic_ratio", "s_pos_par", "num_ratio"]]
    
    return train_data_rougeLTid, train_data_feats, labels_train, dev_data_rougeLTid, dev_data_feats, labels_dev, test_data_rougeLTid, test_data_feats, labels_test


def load_dataset_train(df_dir, sc_type="rougeL", train_data_type="train1"):
    data_type=train_data_type
    print(f"\nLoading {data_type.capitalize()} Data . . ")
    train_data=pd.read_csv(f"/kaggle/input/summarizer-data/{data_type}_set_feats.csv") 
    
    train_data_rougeLTid=pd.read_csv(os.path.join(df_dir,f"{data_type}_data_{sc_type}.csv"))
    train_data_rougeLTid["text_position"] = train_data_rougeLTid.groupby("text_id").cumcount().add(1)
    
    data_stats(train_data_rougeLTid)
    labels_train=train_data_rougeLTid["rougeL"].values
    
    train_data_feats=train_data
    train_data["sentence"]=train_data_rougeLTid["sentence"]

    train_data_feats=train_data_feats[["len", "s_position", "thematic_ratio", "s_pos_par", "num_ratio"]]
    
    return train_data_rougeLTid, train_data_feats, labels_train


    
def load_dataset_test(df_dir, sc_type="rougeL"):
    
    data_type="test"
    print(f"\nLoading {data_type.capitalize()} Data . . ")
    test_data=pd.read_csv(f"/kaggle/input/summarizer-data/{data_type}_set_feats.csv") 
    
    test_data_rougeLTid=pd.read_csv(os.path.join(df_dir,f"{data_type}_data_{sc_type}.csv"))
    test_data_rougeLTid["text_position"] = test_data_rougeLTid.groupby("text_id").cumcount().add(1)
                                                 
    data_stats(test_data_rougeLTid)
    labels_test=test_data_rougeLTid["rougeL"].values
    
    test_data_feats=test_data
    test_data["sentence"]=test_data_rougeLTid["sentence"]
    test_data_feats=test_data_feats[["len", "s_position", "thematic_ratio", "s_pos_par", "num_ratio"]]
    
    return test_data_rougeLTid, test_data_feats, labels_test


# Rouge scores of predicted summaries
def scoring(pred_summary, ref_summary,doc_id):
    pred_summary = re.sub(r'\n+', '', pred_summary)
    
    score_ind={"precision":0, "recall":1, "fmeasure":2}
    r_scorer=rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=True)

    rouge=r_scorer.score(ref_summary,pred_summary)
    
    r1=rouge["rouge1"][score_ind["fmeasure"]]
    r2=rouge["rouge2"][score_ind["fmeasure"]]
    rL=rouge["rougeL"][score_ind["fmeasure"]]
    
    with open(os.path.join(data_dir,"summaResults.txt"), "a", encoding="utf-8") as writer:
        file_str=f"> Document #{doc_id}\n\nPredicted Summary\n{pred_summary}\n\nReference Summary\n{ref_summary}\n\nR1: {r1:.9f} | R2: {r2:.9f} | RL: {rL:.9f}\n\n\n\n"
        writer.write(file_str)
        
    return r1,r2,rL


# functions to create the predicted summary after the sentence scoring
def summarization(sentences):
    sentences = [ str(s) for s in sentences if len(s)>0 ]
    if len(sentences)==0:
        return " "
    
    summary = ".".join(sentences)
    if summary[-1] is not ".":
        summary+="."
    
    return summary


# keep s_num sentences with RougeL score > threshold th to create the predicted summary
def create_summary(df,s_num,th=0.19):
    n_largest=df.groupby(["text_id"])["text_position","sentence","pred_rougeL"].apply(lambda x: x.nlargest(s_num,columns=["pred_rougeL"]).sort_index())
    max_rL=max(n_largest["pred_rougeL"])
    sent = [ n_largest.loc[n_largest["pred_rougeL"].idxmax(),"sentence"] ]
    sent.extend( [ s for s,r in zip(n_largest["sentence"].values, n_largest["pred_rougeL"].values) if r > th and r!=max_rL] )
    summary = summarization( sent)   
    return summary


def get_ref_summary(ref_summaries,doc_id):
    ref_sum=ref_summaries.loc[ref_summaries.text_id==doc_id, "summary"].values
    
    if type(ref_sum)!=str:
        ref_sum=ref_sum[0]
#     if type(ref_sum)!=str:
#         ref_sum=" "
    return ref_sum


def get_preds(model,docX):
    return model.predict(docX)


def doc_summary(model,document,ref_summaries,doc_id,th,s_num=4) :
    global doc_p_bar, tot_r1, tot_r2, tot_rL

    cols=["len", "s_position", "thematic_ratio", "s_pos_par", "num_ratio"]
    feats=document[cols]


    preds=get_preds(model,feats)
    document["pred_rougeL"]=preds.tolist()
    pred_summary=create_summary(document,s_num,th)
    ref_summary=get_ref_summary(ref_summaries,doc_id)
    r1,r2,rL = scoring(pred_summary,ref_summary,doc_id)
    tot_r1.append(r1)
    tot_r2.append(r2)
    tot_rL.append(rL)
    doc_p_bar.postfix[1] = np.mean(tot_r1)
    doc_p_bar.postfix[3] = np.mean(tot_r2)
    doc_p_bar.postfix[5] = np.mean(tot_rL)
    doc_p_bar.postfix[6]["value"] = doc_id
    doc_p_bar.update(1)

    del document
    del feats
    del preds
    del pred_summary
    del ref_summary
    gc.collect()
    
    return r1, r2, rL


# memory clean
# for v in globals():
#     print(v)
#     if str(v) not "__name__":
#         del v
gc.collect()

0

## Essential DataFrames Creation
___

In [ ]:
# test_json_path="/kaggle/input/summarizer-data/test.jsonl"
# json_to_df(test_json_path,"test")

# dev_json_path="/kaggle/input/summarizer-data/dev.jsonl"
# json_to_df(dev_json_path,"dev")

# load saved dataframes and create labels
# train_data_list, dev_data, test_data = text_proc_labels(data_dir)
# train_data_list, dev_data = text_proc_labels(data_dir)

# load dataframes with labels
# df_add_tid(data_dir, df_dir)


FileLinks(".")

## Train, Dev & Test features
___

In [ ]:
# train_data1, train_data2 =  pd.read_csv(f"/kaggle/input/summarizer-data/train1_data_rougeL.csv"), pd.read_csv(f"/kaggle/input/summarizer-data/train2_data_rougeL.csv")
# train_data3, train_data4 = pd.read_csv(f"/kaggle/input/summarizer-data/train3_data_rougeL.csv"), pd.read_csv(f"/kaggle/input/summarizer-data/train4_data_rougeL.csv")
# dev_data = pd.read_csv(f"/kaggle/input/summarizer-data/dev_set.csv")
# test_data = pd.read_csv(f"/kaggle/input/summarizer-data/test_set.csv")

# _____________________________________________________________________________
# feature_df(test_data,data_dir,"test")
# feature_df(dev_data,data_dir,"dev")
# FileLinks(".")

# data_type="train1"
# feature_df(train_data1,data_dir,data_type)
# FileLinks(".")

# data_type="train2"
# feature_df(train_data2,data_dir,data_type)
# FileLinks(".")

# data_type="train3"
# feature_df(train_data3,data_dir,data_type)
# FileLinks(".")

# data_type="train4"
# feature_df(train_data4,data_dir,data_type)
# FileLinks(".")


## Load Train, Dev & Test feature and label datasets
___

In [6]:
score_type="rougeL"
train_data, train_data_feats, labels, dev_data, dev_data_feats, labels_dev, test_data, test_data_feats, labels_test = load_datasets(df_dir, sc_type=score_type)
train_data1, train_data_feats1, labels1 = load_dataset_train(df_dir, sc_type=score_type, train_data_type="train1")
train_data2, train_data_feats2, labels2 = load_dataset_train(df_dir, sc_type=score_type, train_data_type="train2")
# train_data3, train_data_feats3, labels3 = load_dataset_train(df_dir, sc_type=score_type, train_data_type="train3")
# train_data4, train_data_feats4, labels4 = load_dataset_train(df_dir, sc_type=score_type, train_data_type="train4")

FileLinks(".")


Loading Train1 Data . . 
DataFrame shape: (2444760, 5)


            count      mean       std  min      25%       50%       75%  max
chosen                                                                      
0       2318103.0  0.105294  0.120241  0.0  0.05000  0.083333  0.122449  1.0
1        126657.0  0.349503  0.234453  0.0  0.17734  0.306667  0.462428  1.0


Loading Dev Data . . 
DataFrame shape: (1050564, 5)


            count      mean       std  min       25%       50%       75%  max
chosen                                                                       
0       1021110.0  0.109909  0.127365  0.0  0.051948  0.088889  0.128205  1.0
1         29454.0  0.457510  0.297183  0.0  0.216561  0.392666  0.666667  1.0


Loading Test Data . . 
DataFrame shape: (1037722, 5)


            count      mean       std  min       25%       50%       75%  max
chosen                                                                       
0       1008495.0  0.109895  0.127409  0.0  0.051948 

./
  __notebook_source__.ipynb

## Scorer Fine Tuning
___

In [7]:
# grid search - Dev Set Tuning
def estimator_tuning(X, y, estimator=SGDRegressor(), scaler=StandardScaler()):
    model_grid = make_pipeline(scaler, estimator )

    param_grid = {
        "sgdregressor__alpha": [8.192e-10],
        "sgdregressor__tol": [6.4e-5],
        "sgdregressor__epsilon": [3.2e-4],
        "sgdregressor__loss": ["squared_error"],
        "sgdregressor__penalty": ["elasticnet"],
        "sgdregressor__learning_rate": ["adaptive"]
    }
    # "sgdregressor__alpha": 5.0 ** -np.arange(2, 7)
    # "sgdregressor__tol": 10.0 ** -np.arange(2, 7)
    # "sgdregressor__loss": ["squared_error", "huber", "epsilon_insensitive"]
    # sgdregressor__penalty": ["l2", "l1", "elasticnet"]
    # "sgdregressor__learning_rate": ["constant", "optimal", "invscaling", "adaptive"]

    g_search = GridSearchCV(model_grid, param_grid, verbose=9, return_train_score=True, cv=2)
    g_search.fit(X, y)
    
    with open(os.path.join("/kaggle/working/Data",f"{estimator[:-2]}_grid_search_results.txt"), "w", encoding="utf-8" ) as writer:
          writer.write(f"Best ParametersL:\n{g_search.best_params_}\n\n\n{g_search.cv_results_}")

    print(f"Best score: { g_search.best_score_}\nParams: {g_search.best_params_}")
    return g_search.best_estimator_


best_model=estimator_tuning(dev_data_feats, labels_dev)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END sgdregressor__alpha=8.192e-10, sgdregressor__epsilon=0.00032, sgdregressor__learning_rate=adaptive, sgdregressor__loss=squared_error, sgdregressor__penalty=elasticnet, sgdregressor__tol=6.4e-05;, score=(train=0.195, test=0.189) total time=   3.7s
[CV 2/2] END sgdregressor__alpha=8.192e-10, sgdregressor__epsilon=0.00032, sgdregressor__learning_rate=adaptive, sgdregressor__loss=squared_error, sgdregressor__penalty=elasticnet, sgdregressor__tol=6.4e-05;, score=(train=0.190, test=0.194) total time=   3.7s
Best score: 0.19161433549061424
Params: {'sgdregressor__alpha': 8.192e-10, 'sgdregressor__epsilon': 0.00032, 'sgdregressor__learning_rate': 'adaptive', 'sgdregressor__loss': 'squared_error', 'sgdregressor__penalty': 'elasticnet', 'sgdregressor__tol': 6.4e-05}


## Best Scorer
___

In [8]:
def estimator_training(train_data_feats, labels, test_data_feats, labels_test):
    model = make_pipeline(StandardScaler(), SGDRegressor(alpha=8.192e-10, max_iter=1000, tol=6.4e-5, epsilon=3.2e-4, learning_rate="adaptive", loss="squared_error", penalty="elasticnet"))
    model.fit(train_data_feats, labels)

    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    
    print('\n\n> Scores')
    # evaluate model
    test_samples=1000
    # MAE
    scores_mae = cross_val_score(model, test_data_feats.iloc[:test_samples,:] , labels_test[:test_samples], scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    scores_mae = np.absolute(scores_mae)
    print('Mean MAE: %.3f (%.3f)' % (np.mean(scores_mae), np.std(scores_mae)))
    # MSE
    scores_mse = cross_val_score(model, test_data_feats.iloc[:test_samples,:] , labels_test[:test_samples], scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
    scores_mse = np.absolute(scores_mse)
    print('Mean MSE: %.3f (%.3f)' % (np.mean(scores_mse), np.std(scores_mse)))
    # R2    
    scores_r2 = cross_val_score(model, test_data_feats.iloc[:test_samples,:] , labels_test[:test_samples], scoring='r2', cv=cv, n_jobs=-1)
    # scores_r2 = np.absolute(scores_r2)
    print('Mean R2: %.3f (%.3f)' % (np.mean(scores_r2), np.std(scores_r2)))
    
    return model


train_data=pd.DataFrame(np.vstack((train_data1,train_data2)))
train_data_feats=pd.DataFrame(np.vstack((train_data_feats1,train_data_feats2)))
labels=np.concatenate((labels1,labels2),axis=0)
score_type="rougeL"
test_data, test_data_feats, labels_test = load_dataset_test(df_dir, sc_type=score_type)
model=estimator_training(train_data_feats, labels, test_data_feats, labels_test)

pkl_filepath=os.path.join("/kaggle/working/Data","SGD_model.pkl")
with open(pkl_filepath,"wb") as model_writer:
    pickle.dump(model, model_writer)


Loading Test Data . . 
DataFrame shape: (1037722, 5)


            count      mean       std  min       25%       50%       75%  max
chosen                                                                       
0       1008495.0  0.109895  0.127409  0.0  0.051948  0.088889  0.128205  1.0
1         29227.0  0.456524  0.300030  0.0  0.210927  0.389610  0.666667  1.0


> Scores
Mean MAE: 0.069 (0.007)
Mean MSE: 0.016 (0.005)
Mean R2: 0.156 (0.158)


## Evaluation
___

In [9]:
score_type="rougeL"
test_data, test_data_feats, labels_test = load_dataset_test(df_dir, sc_type=score_type)

sc_type="rougeL"
data_type="test"
labels=test_data["rougeL"].values

summaries=pd.read_csv("/kaggle/input/summarizer-data/test_data_rougeL_summaries_grouped.csv")
summaries=pd.DataFrame(summaries["summary"])
summaries["text_id"]=summaries.index

test_data_feats["sentence"]=test_data["sentence"]
test_data_feats["text_id"]=test_data["text_id"]
test_data_feats["text_position"]=test_data["text_position"]

X=test_data_feats
y=pd.DataFrame(labels_test)


Loading Test Data . . 
DataFrame shape: (1037722, 5)


            count      mean       std  min       25%       50%       75%  max
chosen                                                                       
0       1008495.0  0.109895  0.127409  0.0  0.051948  0.088889  0.128205  1.0
1         29227.0  0.456524  0.300030  0.0  0.210927  0.389610  0.666667  1.0


In [10]:
data_dir="/kaggle/working/Data"
open(os.path.join(data_dir,"summaResults.txt"),"w")
%cd /kaggle/working
FileLinks(".")

/kaggle/working


./
  __notebook_source__.ipynb
./Data/
  SGD_model.pkl
  SGDRegressor()_grid_search_results.txt
  summaResults.txt

In [ ]:
cols=["len", "s_position", "thematic_ratio", "s_pos_par", "num_ratio"]
feats=test_data_feats[cols]
preds=model.predict(feats).tolist()
preds_df=pd.DataFrame({"sentence": test_data_feats["sentence"], "text_id": test_data_feats["text_id"], "text_position": test_data_feats["text_position"], "pred_rougeL": preds})

# max sentences to keep -> s_num_max & threshold for sentence rougeL score
threshold=0.5
s_num_max=5
# create predicted summaries
pred_sums=preds_df.groupby("text_id").apply(lambda doc: create_summary(doc,s_num_max,threshold))

# scores of predicted summaries
r_scorer=rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=True)
scores=[ r_scorer.score(ref,pred) for ref,pred in zip(summaries["summary"].values,pred_sums) ]

# score dataframes for visualization
score_df=pd.DataFrame(scores)
score_df["ref_sum"]=summaries["summary"].values
score_df["pred_sum"]=pred_sums
score_df["text_id"]=score_df.index

score_df["rouge1_precision"]=score_df["rouge1"].apply(lambda x: x[0])
score_df["rouge1_recall"]=score_df["rouge1"].apply(lambda x: x[1])
score_df["rouge1_fmeasure"]=score_df["rouge1"].apply(lambda x: x[2])
del score_df["rouge1"]

score_df["rouge2_precision"]=score_df["rouge2"].apply(lambda x: x[0])
score_df["rouge2_recall"]=score_df["rouge2"].apply(lambda x: x[1])
score_df["rouge2_fmeasure"]=score_df["rouge2"].apply(lambda x: x[2])
del score_df["rouge2"]

score_df["rougeL_precision"]=score_df["rougeL"].apply(lambda x: x[0])
score_df["rougeL_recall"]=score_df["rougeL"].apply(lambda x: x[1])
score_df["rougeL_fmeasure"]=score_df["rougeL"].apply(lambda x: x[2])
del score_df["rougeL"]


# store predictions & their respective scores
def log_to_file(row):
    pred_summary = re.sub(r'\n+', '', row['pred_sum'])
    doc_str=f"> Document #{row['text_id']}\n\n#Predicted Summary\n{pred_summary}\n\n#Reference Summary\n{row['ref_sum']}\n\nR1: {row['rouge1_fmeasure']}  |  R2: {row['rouge2_fmeasure']}  |  RL:  {row['rougeL_fmeasure']}\n\n\n\n"
    return doc_str

file_str=score_df.apply(lambda row: log_to_file(row), axis=1)

with open(os.path.join(data_dir,f"summaResults_{data_type}_N{s_num_max}_th{threshold}.txt"),"w",encoding="utf-8") as log_writer:
    for row_str in file_str:
        log_writer.write(row_str)
        
FileLinks(".")

In [ ]:
score_stats=score_df.describe()
score_stats.to_csv(os.path.join(data_dir,f"results_stats_{data_type}_N{s_num_max}_th{threshold}.csv"), header=True, index=True)
score_stats_plt=score_df.describe()
scores_to_plot=score_stats_plt.iloc[1:,1:]
scores_to_plot

## Score Visualization
___

In [ ]:
score_stats_plt=score_df.describe()
scores_to_plot=score_stats_plt.iloc[1:,1:]
data_dir="/kaggle/working/Data"

plot_types=["area", "bar", "box", "kde", "line"]
for kind in plot_types:
    scores_to_plot.plot(kind=kind,figsize=(24,12), grid=True, rot=0)
    plt.savefig(os.path.join(data_dir,f"score_stats_{data_type}_{kind}.png"),dpi=300)

FileLinks(".")

## 📚 References

1. [Named Entity Recognition (NER) with TensorflowNamed Entity Recognition (NER) with Tensorflow](https://www.kaggle.com/code/naseralqaydeh/named-entity-recognition-ner-with-tensorflow)
2. [Extractive Summarization using Deep LearningExtractive Summarization using Deep Learning](https://arxiv.org/pdf/1708.04439v1.pdf)
3. [NLTK](https://www.bogotobogo.com/python/NLTK/Stemming_NLTK.php)
4. [Text Features Library](https://github.com/pmbaumgartner/text-feat-lib/tree/master/notebooks)
5. [SummaRuNNer: a recurrent neural network based sequence model for extractive summarization of documents](https://arxiv.org/abs/1611.04230)
6. [Applying Text Classification Using Logistic Regression](https://medium.com/analytics-vidhya/applying-text-classification-using-logistic-regression-a-comparison-between-bow-and-tf-idf-1f1ed1b83640)
7. [Text Summarization in Python using Extractive method (including end-to-end implementation)](https://medium.com/analytics-vidhya/text-summarization-in-python-using-extractive-method-including-end-to-end-implementation-2688b3fd1c8c)
8. [An intro to ROUGE, and how to use it to evaluate summaries](https://www.freecodecamp.org/news/what-is-rouge-and-how-it-works-for-evaluation-of-summaries-e059fb8ac840/)
9. [A Comparative Study of Classifiers for Extractive Text Summarization](https://www.researchgate.net/profile/Anshuman-Pattanaik/publication/340111386_A_Comparative_Study_of_Classifiers_for_Extractive_Text_Summarization/links/5f81d18792851c14bcbc59e1/A-Comparative-Study-of-Classifiers-for-Extractive-Text-Summarization.pdf)

